In [ ]:
import cv2
import os
import time
import pandas as pd
import glob

# --- Escenario A: Creación de un Dataset Propio ---

def create_new_dataset(output_dir='dataset_propio'):
    """
    Inicia una herramienta interactiva para capturar y etiquetar frames
    desde la cámara web para crear un dataset propio.
    """
    # Definir las clases y crear directorios
    clases = {'a': 'atencion', 'd': 'distraccion', 'f': 'fatiga'}
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for clase in clases.values():
        class_dir = os.path.join(output_dir, clase)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
            print(f"Directorio creado: {class_dir}")

    # Iniciar captura de video
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se puede abrir la cámara.")
        return

    print("\n--- Herramienta de Creación de Dataset Iniciada ---")
    print("Enfóquese en la ventana de la cámara.")
    print("Presione 'a' para guardar un frame como 'Atención'.")
    print("Presione 'd' para guardar un frame como 'Distracción'.")
    print("Presione 'f' para guardar un frame como 'Fatiga'.")
    print("Presione 'q' para salir.")

    counts = {clase: 0 for clase in clases.values()}

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: No se pudo leer el frame.")
            break

        # Mostrar el frame
        cv2.imshow('Creación de Dataset - Presione (a, d, f) para etiquetar, (q) para salir', frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            print("Saliendo...")
            break

        if key in [ord(k) for k in clases.keys()]:
            clase_key = chr(key)
            clase_nombre = clases[clase_key]
            class_dir = os.path.join(output_dir, clase_nombre)

            # Guardar el frame
            counts[clase_nombre] += 1
            filename = f"{clase_nombre}_{int(time.time())}_{counts[clase_nombre]}.jpg"
            filepath = os.path.join(class_dir, filename)
            cv2.imwrite(filepath, frame)

            print(f"Guardado: {filepath}")

            # Feedback visual
            cv2.putText(frame, f"GUARDADO: {clase_nombre.upper()}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Creación de Dataset - Presione (a, d, f) para etiquetar, (q) para salir', frame)
            cv2.waitKey(500) # Pausa breve para mostrar el feedback

    cap.release()
    cv2.destroyAllWindows()
    print("--- Creación de Dataset Finalizada ---")
    print(f"Resumen de frames guardados: {counts}")


# --- Escenario B: Carga de un Dataset Existente (Ej. DAiSEE) ---

def load_existing_dataset(daisee_dir='DAiSEE'):
    """
    Simula la carga y procesamiento de un dataset existente como DAiSEE.
    DAiSEE debe descargarse por separado y esta función espera
    su estructura de carpetas (videos y etiquetas CSV).

    Referencia DAiSEE: Gopalan, A., et al. (2016). DAiSEE.
    """
    print(f"\n--- Cargando Dataset Existente (Simulación DAiSEE) ---")

    # Rutas esperadas (ejemplo)
    labels_path = os.path.join(daisee_dir, 'Labels', 'AllLabels.csv')
    videos_dir = os.path.join(daisee_dir, 'DataSet', 'Videos')

    if not os.path.exists(labels_path) or not os.path.exists(videos_dir):
        print(f"Error: No se encontró el dataset DAiSEE en la ruta '{daisee_dir}'.")
        print("Por favor, descargue el dataset y ajuste las rutas.")
        return

    # Cargar etiquetas
    try:
        labels_df = pd.read_csv(labels_path)
        print(f"Etiquetas cargadas: {len(labels_df)} registros encontrados.")

        # Mapeo de estados afectivos de DAiSEE (ejemplo)
        # DAiSEE usa (0=Very Low, 1=Low, 2=Medium, 3=High)
        # Podemos binarizar: 0,1 -> 'No Atencion', 2,3 -> 'Atencion'
        # O usar 'Boredom', 'Confusion', 'Frustration'

        print("Mostrando 5 primeras etiquetas de 'Engagement':")
        print(labels_df[['ClipID', 'Engagement']].head())

        # Procesar videos (ejemplo con el primer video)
        if not labels_df.empty:
            first_clip_id = labels_df.iloc[0]['ClipID']
            video_path = os.path.join(videos_dir, f"{first_clip_id}.avi")

            if os.path.exists(video_path):
                print(f"Procesando video de ejemplo: {video_path}")
                cap = cv2.VideoCapture(video_path)
                frame_count = 0
                while cap.isOpened() and frame_count < 50: # Solo 50 frames
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Aquí iría la lógica de pre-procesamiento
                    # frame_procesado = preprocess(frame)

                    frame_count += 1
                cap.release()
                print(f"Video {first_clip_id} procesado (simulado).")
            else:
                print(f"Video {first_clip_id}.avi no encontrado.")

    except Exception as e:
        print(f"Error al procesar el dataset DAiSEE: {e}")

# --- Ejecución Principal ---
if __name__ == "__main__":

    print("Seleccione una opción:")
    print("1: Iniciar herramienta interactiva para CREAR un dataset propio.")
    print("2: Simular la carga de un dataset EXISTENTE (DAiSEE).")

    choice = input("Ingrese 1 o 2: ")

    if choice == '1':
        create_new_dataset()
    elif choice == '2':
        # Asegúrese de tener el dataset DAiSEE en una carpeta llamada 'DAiSEE'
        # o ajuste la ruta
        load_existing_dataset(daisee_dir='DAiSEE')
    else:
        print("Opción no válida.")